In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-in-csv/mnist_test.csv
/kaggle/input/mnist-in-csv/mnist_train.csv


## Classificador Linear de Mínimos Quadrados

In [ ]:
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

#Divisão dos dados em treino e teste
y_train = df_train.iloc[:,0].values
x_train = df_train.iloc[:,1:].values
y_test = df_test.iloc[:,0].values
x_test = df_test.iloc[:,1:].values

#y_train = y_train.reshape(-1,1)
#x_train = x_train / 255.0 #Normalização
#x_test = x_test / 255.0 #Normalização

#Criação de uma função para binarizar os rótulos
def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

# Codificar os rótulos de treino e teste
y_train_one_hot = one_hot_encode(y_train, 10)
y_test_one_hot = one_hot_encode(y_test, 10)

# Adicionar uma coluna do coeficiente em B0(intercepto)
x_train_bias = np.c_[np.ones((x_train.shape[0], 1)), x_train]
x_test_bias = np.c_[np.ones((x_test.shape[0], 1)), x_test]

# Regularização
lambda_identity = 1e-3 * np.eye(x_train_bias.shape[1])  # Para regularização

# Resolver a equação normal: W = (X^T * X)^-1 * X^T * Y
B = np.linalg.inv(x_train_bias.T @ x_train_bias + lambda_identity) @ x_train_bias.T @ y_train_one_hot
#W = np.linalg.pinv(x_train_bias.T) @ y_train_one_hot

y_pred = x_test_bias @ B

y_pred_labels = np.argmax(y_pred, axis=1)

from sklearn.metrics import accuracy_score, confusion_matrix

# Calcular a precisão
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Acurácia: {accuracy * 100:.2f}%")

Acurácia: 86.03%


A aplicação do modelo de classificação linear usando os mínimos quadrados gerou uma acurácia de 86%, enquanto o modelo a ser comparado gerou uma acurácia de 88%. Em seguida outros modelos serão comparados com o modelo em questão para analisar a acurácia dos modelos.

Durante os testes com o modelo, foi verificado que a matriz era singular, assim, para evitar o problema de singularidade foi utilizado o metodo de regularização de Thikonov.

Como o modelo a ser comparado na questão não utiliza pre processamento os dados MNIST também não sofreu nenhuma modificação em seus dados originais.

Para fins de estudo, foi aplicada uma normalização nos dados para verificar  se ocorreria alguma melhoria ao aplicar a normalização dos dados, porém o resultado permaneceu o mesmo.

Foi realizada a transformação dos dados de saída para o sistema multiclasse. Como o sistema MNIST possui 10 classe, 0 até 9, os rótulos são transformados para um vetor binário  no qual apenas a posição referente ao valor correto recebe o valor de 1 e os outros valores recebem o valor 0.



## Adaline com Função de Ativação Sigmoidal

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Carregar os dataframes
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar os rótulos e dados de treino
y_train = df_train.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_train = df_train.iloc[:, 1:].values #/ 255.0  # Normalização (dividindo por 255.0)

# Separar os rótulos e dados de teste
y_test = df_test.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_test = df_test.iloc[:, 1:].values #/ 255.0  # Normalização (dividindo por 255.0)

# Converter rótulos para NumPy arrays e redimensioná-los
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Codificação one-hot para os rótulos (necessário para 10 classificadores binários)
encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = encoder.fit_transform(y_train)
y_test_one_hot = encoder.transform(y_test)

# Adicionar uma coluna de 1s para o bias em X
X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]  # Adicionando coluna de 1s
X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]  # Adicionando coluna de 1s

# Função sigmoide
def sigmoid(z):
    """Função sigmoide."""
    return 1 / (1 + np.exp(-z))

# Função para treinar MADALINE com função sigmoide
def train_madaline_sigmoid(X, y, epochs=1000, lr=0.01):
    n_samples, n_features = X.shape
    n_classes = y.shape[1]

    # Inicializar os pesos (incluindo o bias como parte do vetor de pesos)
    weights = np.random.rand(n_classes, n_features)  # Pesos para cada classe

    for epoch in range(epochs):
        # Calcular a saída para todas as amostras
        z_input = np.dot(X, weights.T)  # Saídas lineares para todas as amostras
        z = sigmoid(z_input)  # Aplicar função sigmoide

        # Calcular o erro (diferença entre o esperado e o previsto)
        error = y - z

        # Atualizar os pesos com base no erro
        for j in range(n_classes):
            # Atualizar pesos apenas para as classes onde houve erro
            mask = (y[:, j] > 0) & (z[:, j] <= 0.5) | (y[:, j] <= 0) & (z[:, j] > 0.5)
            weights[j] += lr * np.dot(mask * error[:, j], X)  # Atualizar pesos

        # Imprimir erro por época
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Error = {np.sum(np.square(error)):.4f}") #Erro quadrático

    return weights

# Treinar o modelo MADALINE com função sigmoide
weights = train_madaline_sigmoid(X_train_bias, y_train_one_hot, epochs=1000, lr=0.01)

# Fazer previsões no conjunto de teste
y_pred_prob = sigmoid(X_test_bias @ weights.T)  # O bias está incorporado

# Converter probabilidades para rótulos (classe com maior probabilidade)
y_pred_labels = np.argmax(y_pred_prob, axis=1)

# Calcular a acurácia
accuracy = accuracy_score(y_test.flatten(), y_pred_labels)
print(f"Acurácia: {accuracy * 100:.2f}%")


Epoch 0: Error = 540000.0000


/tmp/ipykernel_36/701434130.py:34: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Epoch 100: Error = 31350.0000
Epoch 200: Error = 25976.0000
Epoch 300: Error = 25850.0000
Epoch 400: Error = 20406.0000
Epoch 500: Error = 28022.0000
Epoch 600: Error = 19758.0000
Epoch 700: Error = 21621.0000
Epoch 800: Error = 19882.0000
Epoch 900: Error = 18443.0000
Acurácia: 86.58%


A aplicação do modelo MADALINE com função de ativação sigmoidal apresentou uma Acurácia de valor próximo ao do modelo linear, com valor de 86,58%. Para o desenvolvimento do modelo foi mantida a não aplicação do pré processamento dos dados, a fim de comparação com o modelo linear e a aplicação dele e do modelo em questão com o PCA.

Durante o desenvolvimento do modelo foram realizados testes com o embaralhamento dos dados, porém os resultados apresentados se mantiveram semelhante ao modelo testado sem o embaralhamento. Durante a aplicação da variação da quantidade de épocas foi observada que os melhores valores da acurácia nos dados de teste foram a partir de 500 épocas. As taxas de aprendizagem testadas foram de 0.1, 0.05 e 0.01, o melhor resultado apresentado foi com a taxa de 0.01.

## PCA com modelo Adaline e função de ativação Sigmoidal

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

# Carregar os dataframes
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar os rótulos e dados de treino
y_train = df_train.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_train = df_train.iloc[:, 1:].values / 255.0  # Normalização (dividindo por 255.0)

# Separar os rótulos e dados de teste
y_test = df_test.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_test = df_test.iloc[:, 1:].values / 255.0  # Normalização (dividindo por 255.0)

# Converter rótulos para NumPy arrays e redimensioná-los
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Codificação one-hot para os rótulos (necessário para 10 classificadores binários)
encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = encoder.fit_transform(y_train)
y_test_one_hot = encoder.transform(y_test)

# Aplicar PCA para reduzir a dimensionalidade (preservando 95% da variância)
pca = PCA(n_components=0.99)  # ou pode escolher um número fixo, por exemplo, n_components=50
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Adicionar uma coluna de 1s para o bias em X
X_train_bias = np.c_[np.ones((X_train_pca.shape[0], 1)), X_train_pca]  # Adicionando coluna de 1s
X_test_bias = np.c_[np.ones((X_test_pca.shape[0], 1)), X_test_pca]  # Adicionando coluna de 1s

EQM = []
# Função sigmoide
def sigmoid(z):
    """Função sigmoide."""
    return 1 / (1 + np.exp(-z))

# Função para treinar MADALINE com função sigmoide
def train_madaline_sigmoid(X, y, epochs=1000, lr=0.01):
    n_samples, n_features = X.shape
    n_classes = y.shape[1]

    # Inicializar os pesos (incluindo o bias como parte do vetor de pesos)
    weights = np.random.rand(n_classes, n_features)  # Pesos para cada classe

    for epoch in range(epochs):
        # Calcular a saída para todas as amostras
        z_input = np.dot(X, weights.T)  # Saídas lineares para todas as amostras
        z = sigmoid(z_input)  # Aplicar função sigmoide

        # Calcular o erro (diferença entre o esperado e o previsto)
        error = y - z

        # Atualizar os pesos com base no erro
        for j in range(n_classes):
            # Atualizar pesos apenas para as classes onde houve erro
            mask = (y[:, j] > 0) & (z[:, j] <= 0.5) | (y[:, j] <= 0) & (z[:, j] > 0.5)
            weights[j] += lr * np.dot(mask * error[:, j], X)  # Atualizar pesos

        # Imprimir erro por época
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Error = {np.sum(np.square(error)):.4f}") #Erro quadrático
        EQM.append(np.sum(np.square(error)))
    return weights, EQM

# Treinar o modelo MADALINE com função sigmoide
weights, EQM = train_madaline_sigmoid(X_train_bias, y_train_one_hot, epochs=1000, lr=0.01)

# Fazer previsões no conjunto de teste
y_pred_prob = sigmoid(X_test_bias @ weights.T)  # O bias está incorporado

# Converter probabilidades para rótulos (classe com maior probabilidade)
y_pred_labels = np.argmax(y_pred_prob, axis=1)

# Calcular a acurácia
accuracy = accuracy_score(y_test.flatten(), y_pred_labels)
print(f"Acurácia: {accuracy * 100:.2f}%")

Epoch 0: Error = 255943.6697


/tmp/ipykernel_36/801356226.py:41: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Epoch 100: Error = 22270.6747
Epoch 200: Error = 16235.2463
Epoch 300: Error = 20172.1377
Epoch 400: Error = 20757.9162
Epoch 500: Error = 25312.6827
Epoch 600: Error = 19081.4735
Epoch 700: Error = 18625.4405
Epoch 800: Error = 13038.1918
Epoch 900: Error = 19762.3195
Acurácia: 88.48%


A aplicação do PCA com uma retenção de 99% da variância no modelo Adaline trouxe uma melhoria na Acurácia, elevando de aproximadamente 86% para 88,48%. Foram testadas menores retenções como 90% e 85%, porém em ambos os casos o modelo teve uma perda na acurácia se comparado com o modelo Adaline sem a aplicação do PCA, ficando entre 84% e 82%.

Uma observação constatada foi que durante os testes o valor do erro tem uma rápida queda, porém após esse primeiro momento o ocorre uma oscilação do erro, esse mesmo fato não ocorre sem o uso do PCA.

## PCA + Classificador linear com Minimos Quadrados

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Carregar os dataframes
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar os rótulos e dados de treino
y_train = df_train.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_train = df_train.iloc[:, 1:].values / 255.0  # As demais colunas contêm os dados (normalização)

# Separar os rótulos e dados de teste
y_test = df_test.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_test = df_test.iloc[:, 1:].values / 255.0  # As demais colunas contêm os dados (normalização)

# Aplicar PCA para reduzir a dimensionalidade (preservando 95% da variância)
pca = PCA(n_components=0.94)  # PCA manterá componentes suficientes para explicar 95% da variância
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Codificar os rótulos de treino e teste com one-hot encoding
def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

y_train_one_hot = one_hot_encode(y_train, 10)
y_test_one_hot = one_hot_encode(y_test, 10)

# Adicionar uma coluna de bias aos dados com PCA
X_train_bias = np.c_[np.ones((X_train_pca.shape[0], 1)), X_train_pca]
X_test_bias = np.c_[np.ones((X_test_pca.shape[0], 1)), X_test_pca]

# Regularização
lambda_identity = 1e-3 * np.eye(X_train_bias.shape[1])  # Para regularização (lambda = 0.001)

# Resolver a equação normal: W = (X^T * X + lambda*I)^-1 * X^T * Y
W = np.linalg.inv(X_train_bias.T @ X_train_bias + lambda_identity) @ X_train_bias.T @ y_train_one_hot

# Fazer previsões no conjunto de teste
y_pred = X_test_bias @ W

# Converter as probabilidades preditas para rótulos (classe com maior probabilidade)
y_pred_labels = np.argmax(y_pred, axis=1)

# Calcular a precisão
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Acurácia: {accuracy * 100:.2f}%")

# Matriz de confusão
#conf_matrix = confusion_matrix(y_test, y_pred_labels)
#print("Matriz de confusão:")
#print(conf_matrix)


Acurácia: 86.14%


A aplicação do PCA nos dados de entrada juntamente com a aplicação do modelo de classificação linear teve uma melhora na precisão em relação ao modelo sem PCA. Para isso, a retenção da variância que apresentou o melhor resultado foi de 94%.

| MODELOS | Acurácia |
|------------|-------|
| Classificador Linear | 86,03%    |
| Madaline      | 86,58%    |
| PCA + Classificador Linear | 86,14%    |
| PCA + Madaline      | 88,48%    |

Observando os modelos e comparado ao apresentado na questão, o modelo que apresentou resultado de valor superior foi o modelo PCA + Madaline, com uma Acurácia de 88,48%, enquanto o modelo de comparação tem um valor de 88%.

## MLP com neurônios ocultos definidos por PCA + Heuristica + random search

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Carregar o dataset MNIST
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar rótulos e dados
y_train = df_train.iloc[:, 0].values
X_train = df_train.iloc[:, 1:].values #/ 255.0  # Normalizar
y_test = df_test.iloc[:, 0].values
X_test = df_test.iloc[:, 1:].values #/ 255.0

# Codificação one-hot dos rótulos
y_train_one_hot = to_categorical(y_train, 10)
y_test_one_hot = to_categorical(y_test, 10)

# Escalar os dados
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Definir função para criar MLP com número variável de neurônios ocultos
def create_mlp(hidden_neurons, input_shape, output_shape):
    model = Sequential()
    model.add(Dense(hidden_neurons, input_dim=input_shape, activation='sigmoid'))  # Camada oculta
    model.add(Dense(output_shape, activation='softmax'))  # Camada de saída (10 classes)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

A efeito de comparação com a questão, que não utiliza pre processamento,
os modelos testados seguem o mesmo padrão.

O modelo foi desenvolvido utilizando a biblioteca Tensorflow com uma camada oculta que possui a função de ativação sigmoidal, com o intuito de introduzir não linearidade ao modelo, e a camada de saída com a função de ativação
softmax, ela é utilizada para modelos de classificação multiclasse para a transformação da saída em uma distribuição de probabilidade.

Durante os testes com os modelos foram utilizados os otimizadores SGD e Adam, a utilização do otimizador Adam foi mantido pois o mesmo elevava a acurácia em 10% quando comparado ao SGD.


In [ ]:
# Teste 1: Heurística básica - Regra do valor médio
input_dim = X_train.shape[1]
output_dim = 10  # Número de classes (10 dígitos)

neurons_heuristic = input_dim + output_dim// 2
model_heuristic = create_mlp(neurons_heuristic, input_dim, output_dim)
history_heuristic = model_heuristic.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.8490 - loss: 0.0239 - val_accuracy: 0.9110 - val_loss: 0.0143
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9114 - loss: 0.0143 - val_accuracy: 0.9258 - val_loss: 0.0119
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9213 - loss: 0.0125 - val_accuracy: 0.9233 - val_loss: 0.0122
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9244 - loss: 0.0118 - val_accuracy: 0.9239 - val_loss: 0.0116
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9284 - loss: 0.0111 - val_accuracy: 0.9208 - val_loss: 0.0119
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9297 - loss: 0.0109 - val_accuracy: 0.9303 - val_loss: 0.0106
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9322 - loss: 0.0103 - val_accuracy: 0.9256 - val_loss: 0.0113
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9283 - loss: 0

In [ ]:
# Teste 1: Heurística básica - Regra da raiz quadrada
input_dim = X_train.shape[1]
output_dim = 10  # Número de classes (10 dígitos)

neurons_heuristic = round(np.sqrt(input_dim + output_dim))
model_heuristic = create_mlp(neurons_heuristic, input_dim, output_dim)
history_heuristic = model_heuristic.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6364 - loss: 0.0526 - val_accuracy: 0.8443 - val_loss: 0.0252
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8549 - loss: 0.0230 - val_accuracy: 0.8723 - val_loss: 0.0208
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8667 - loss: 0.0206 - val_accuracy: 0.8772 - val_loss: 0.0193
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8698 - loss: 0.0198 - val_accuracy: 0.8780 - val_loss: 0.0186
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8751 - loss: 0.0189 - val_accuracy: 0.8799 - val_loss: 0.0183
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8711 - loss: 0.0192 - val_accuracy: 0.8837 - val_loss: 0.0174
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8794 - loss: 0.0181 - val_accuracy: 0.8855 - val_loss: 0.0174
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8809 - loss: 0.0180 - 

In [ ]:
# Teste 1: Heurística básica - Regra de Kolgomorov
input_dim = X_train.shape[1]
output_dim = 10  # Número de classes (10 dígitos)

neurons_heuristic = 2 *input_dim + 1
model_heuristic = create_mlp(neurons_heuristic, input_dim, output_dim)
history_heuristic = model_heuristic.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.8040 - loss: 0.0289 - val_accuracy: 0.9142 - val_loss: 0.0133
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9119 - loss: 0.0140 - val_accuracy: 0.9258 - val_loss: 0.0116
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9176 - loss: 0.0127 - val_accuracy: 0.9192 - val_loss: 0.0123
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9219 - loss: 0.0118 - val_accuracy: 0.9260 - val_loss: 0.0112
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9280 - loss: 0.0111 - val_accuracy: 0.9295 - val_loss: 0.0108
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9279 - loss: 0.0109 - val_accuracy: 0.9350 - val_loss: 0.0100
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.9289 - loss: 0.0108 - val_accuracy: 0.9330 - val_loss: 0.0103
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9362 -

In [ ]:
# Teste 2: Baseado em PCA para estimar o número de neurônios ocultos
pca = PCA(n_components=0.95)  # Retenha 95% da variância
pca.fit(X_train)
n_pca_components = pca.n_components_  # Número de componentes principais

model_pca = create_mlp(n_pca_components, input_dim, output_dim)
history_pca = model_pca.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7895 - loss: 0.0329 - val_accuracy: 0.8995 - val_loss: 0.0167
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8990 - loss: 0.0165 - val_accuracy: 0.9074 - val_loss: 0.0149
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8997 - loss: 0.0155 - val_accuracy: 0.9110 - val_loss: 0.0139
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9056 - loss: 0.0146 - val_accuracy: 0.9096 - val_loss: 0.0139
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9103 - loss: 0.0138 - val_accuracy: 0.9152 - val_loss: 0.0132
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9120 - loss: 0.0134 - val_accuracy: 0.9172 - val_loss: 0.0127
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9169 - loss: 0.0127 - val_accuracy: 0.9187 - val_loss: 0.0124
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9220 - loss: 0.0120 - 

In [ ]:
# Teste 3: Busca exaustiva - testar diferentes números de neurônios ocultos
best_accuracy = 0
best_neurons = 0
for neurons in range(50, 401, 50):  # Testar de 50 a 400 neurônios ocultos
    print(f"Testando com {neurons} neurônios ocultos.")
    model = create_mlp(neurons, input_dim, output_dim)
    history = model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot), verbose=0)
    accuracy = history.history['val_accuracy'][-1]
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_neurons = neurons
    print(f"Validação final com {neurons} neurônios: Acurácia = {accuracy:.4f}")

print(f"Melhor número de neurônios ocultos após busca exaustiva: {best_neurons} com acurácia de {best_accuracy:.4f}")


Testando com 50 neurônios ocultos.
Validação final com 50 neurônios: Acurácia = 0.9158
Testando com 100 neurônios ocultos.
Validação final com 100 neurônios: Acurácia = 0.9125
Testando com 150 neurônios ocultos.
Validação final com 150 neurônios: Acurácia = 0.9252
Testando com 200 neurônios ocultos.
Validação final com 200 neurônios: Acurácia = 0.9250
Testando com 250 neurônios ocultos.
Validação final com 250 neurônios: Acurácia = 0.9175
Testando com 300 neurônios ocultos.
Validação final com 300 neurônios: Acurácia = 0.9243
Testando com 350 neurônios ocultos.
Validação final com 350 neurônios: Acurácia = 0.9347
Testando com 400 neurônios ocultos.
Validação final com 400 neurônios: Acurácia = 0.9277
Melhor número de neurônios ocultos após busca exaustiva: 350 com acurácia de 0.9347


O cálculo para a obtenção do valor correto de neurônios em uma camada oculta pode ser realizado de diversas formas, como observado nos testes acima. Pode-se considerar que esses cálculos servem como parâmetro inicial da quantidade de neurônios.

Observando os testes realizados chegou ao resultado de melhor acurácia para a regra de Kolgomorov, esta calcula o numero de nuerônios na camada oculta como 2 vezes o número de entradas na rede mais 1, essa aplicação chegou a uma precisão de 93,58%.

A tabela abaixo mostra os resultados para cada modelo testado com diferentes ténicas de obtenção da quantidade de neurônio.

| MODELOS | Acurácia |
|------------|-------|
| Regra do valor médio | 92,75%    |
| Regra da raiz quadrada      | 89,20%    |
| Kolgomorov | 93,58%    |
| PCA       | 92,63%    |
| Busca Exaustiva (250 neurônios)       | 93,47%    |

Apesar disso, o resultado dessas técnicas ainda ficaram abaixo do valor da precisão do modelo utilizado como comparação na questão, que possui o valor de 95,3% nos dados de teste.

## PCA+MLP com q neurônios ocultos


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Carregar o dataset MNIST
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar rótulos e dados
y_train = df_train.iloc[:, 0].values
X_train = df_train.iloc[:, 1:].values  # Dados de entrada
y_test = df_test.iloc[:, 0].values
X_test = df_test.iloc[:, 1:].values  # Dados de entrada

# Codificação one-hot dos rótulos
y_train_one_hot = to_categorical(y_train, 10)
y_test_one_hot = to_categorical(y_test, 10)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar PCA para retenção de 95% da variância
pca = PCA(0.95)  # Manter 95% da variância
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Verificar a quantidade de componentes principais retidas
print(f"Número de componentes principais retidas: {X_train_pca.shape[1]}")

# Definir função para criar MLP com número variável de neurônios ocultos
def create_mlp(hidden_neurons, input_shape, output_shape):
    model = Sequential()
    model.add(Dense(hidden_neurons, input_dim=input_shape, activation='sigmoid'))  # Camada oculta
    model.add(Dense(output_shape, activation='softmax'))  # Camada de saída (10 classes)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Teste 1: Heurística básica - Regra de Kolmogorov
input_dim = X_train_pca.shape[1]  # Atualizar a dimensão de entrada para as componentes principais
output_dim = 10  # Número de classes (10 dígitos)

neurons_heuristic = 2 * input_dim + 1
model_heuristic = create_mlp(neurons_heuristic, input_dim, output_dim)
history_heuristic = model_heuristic.fit(X_train_pca, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test_pca, y_test_one_hot))

# Avaliar o modelo
loss, accuracy = model_heuristic.evaluate(X_test_pca, y_test_one_hot)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Número de componentes principais retidas: 331


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8557 - loss: 0.0216 - val_accuracy: 0.9305 - val_loss: 0.0106
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9377 - loss: 0.0096 - val_accuracy: 0.9401 - val_loss: 0.0090
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9537 - loss: 0.0072 - val_accuracy: 0.9503 - val_loss: 0.0076
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9643 - loss: 0.0056 - val_accuracy: 0.9558 - val_loss: 0.0066
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9729 - loss: 0.0045 - val_accuracy: 0.9608 - val_loss: 0.0060
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9788 - loss: 0.0034 - val_accuracy: 0.9630 - val_loss: 0.0058
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9833 - loss: 0.0027 - val_accuracy: 0.9667 - val_loss: 0.0051
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9866 - loss: 0.0023 - 

Ao observar a aplicação do modelo de PCA + MLP com o número de neurônios ocultos definidos através da regra de Kolgomorov e a aplicação em 10 epocas, nota-se a obtenção de uma precisão de  aproximadamente 96,86%, resultado superior ao modelo utilizado como comparação da questão 1, no valor de 95,3%.

## Autoencoder + Adaline c/ Função Sigmoidal

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Carregar os dataframes
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar os rótulos e dados de treino
y_train = df_train.iloc[:, 0].values  # A primeira coluna contém os rótulos
X_train = df_train.iloc[:, 1:].values / 255.0  # Normalização
X_test = df_test.iloc[:, 1:].values / 255.0  # Normalização

# Separar os rótulos e dados de teste
y_test = df_test.iloc[:, 0].values  # A primeira coluna contém os rótulos

# Codificação one-hot para os rótulos
encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

# Definir o autoencoder
def create_autoencoder(input_shape, encoding_dim):
    input_layer = Input(shape=(input_shape,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_shape, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)  # Modelo encoder
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder, encoder

# Definir dimensão de codificação
encoding_dim = 256  # Dimensão do espaço latente

# Criar e treinar o autoencoder e o encoder
autoencoder, encoder_model = create_autoencoder(X_train.shape[1], encoding_dim)
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_data=(X_test, X_test))

# Obter as representações codificadas
X_train_encoded = encoder_model.predict(X_train)
X_test_encoded = encoder_model.predict(X_test)

# Confirmar as formas das saídas codificadas
print(f'X_train_encoded shape: {X_train_encoded.shape}')  # Deve ser (num_samples, 64)
print(f'X_test_encoded shape: {X_test_encoded.shape}')    # Deve ser (num_samples, 64)

# Função sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Função para treinar o modelo MADALINE com função de ativação sigmoide
def train_madaline_sigmoid(X, y, epochs=1000, lr=0.01):
    n_samples, n_features = X.shape
    n_classes = y.shape[1]

    # Inicializar os pesos
    weights = np.random.rand(n_classes, n_features)

    for epoch in range(epochs):
        z_input = np.dot(X, weights.T)  # Saídas lineares
        z = sigmoid(z_input)  # Função sigmoide

        # Calcular o erro
        error = y - z

        # Atualizar os pesos
        for j in range(n_classes):
            mask = (y[:, j] > 0) & (z[:, j] <= 0.5) | (y[:, j] <= 0) & (z[:, j] > 0.5)
            weights[j] += lr * np.dot(mask * error[:, j], X)  # Atualizar pesos

        # Imprimir erro por época
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Error = {np.sum(np.square(error)):.4f}")

    return weights

# Treinar o modelo MADALINE com dados codificados
weights = train_madaline_sigmoid(X_train_encoded, y_train_one_hot, epochs=1000, lr=0.1)

# Fazer previsões no conjunto de teste
y_pred_prob = sigmoid(X_test_encoded @ weights.T)
y_pred_labels = np.argmax(y_pred_prob, axis=1)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Acurácia: {accuracy * 100:.2f}%")


Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.2805 - val_loss: 0.1106
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.1032 - val_loss: 0.0857
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0837 - val_loss: 0.0766
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0761 - val_loss: 0.0723
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0722 - val_loss: 0.0701
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0702 - val_loss: 0.0687
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0689 - val_loss: 0.0678
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0681 - val_loss: 0.0671
Epoch 9/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0674 - val_loss: 0.0666
Epoch 10/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0667 - val_loss: 0.0663
Epoch 11/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0665 - val_loss: 0.0659
Epoch 12/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 

/tmp/ipykernel_37/917502392.py:52: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 100: Error = 36102.9937
Epoch 200: Error = 32336.8165
Epoch 300: Error = 42714.9516
Epoch 400: Error = 16907.9999
Epoch 500: Error = 19831.2083
Epoch 600: Error = 21768.0000
Epoch 700: Error = 23233.9889
Epoch 800: Error = 20082.0058
Epoch 900: Error = 16976.0000
Acurácia: 86.76%


Foram realizadas modificações na dimensão do espaço latente do autoencoder, utilizando 64, 128 e 256, porém nenhum dos casos surtiu efeito positivo na precisão do modelo.

Nos casos com valor 64 e 128 ocorreram uma queda na precisão, que no modelo multi Adaline tem uma precisão de 86%, no modelo PCA + multi Adaline com uma precisão de 88%, e agora tiveram resultados de aproximadamente 79% e 82%, respectivamente.

O modelo utilizando o valor latente apresentou um resultado de Acurácia maior, porém próximo ao modelo MADALINE sem qualquer uso de pré processamento.

## Autoencoder + MLP

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

# Carregar o dataset MNIST a partir de arquivos CSV
df_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
df_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

# Separar rótulos e dados
y_train = df_train.iloc[:, 0].values  # A primeira coluna contém os rótulos (dígitos)
X_train = df_train.iloc[:, 1:].values  # As colunas subsequentes contêm os dados de entrada (imagens achatadas)
y_test = df_test.iloc[:, 0].values  # A primeira coluna contém os rótulos (dígitos)
X_test = df_test.iloc[:, 1:].values  # As colunas subsequentes contêm os dados de entrada (imagens achatadas)

# Normalizar os dados para o intervalo [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Codificação one-hot dos rótulos (necessária para classificação)
y_train_one_hot = to_categorical(y_train, 10)
y_test_one_hot = to_categorical(y_test, 10)

# Definir o autoencoder
def create_autoencoder(input_shape, encoding_dim):
    input_layer = Input(shape=(input_shape,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_shape, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)  # Aqui definimos o modelo encoder
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder, encoder

# Definir dimensão de codificação
encoding_dim = 64  # Dimensão do espaço latente

# Criar e treinar o autoencoder e o encoder
autoencoder, encoder = create_autoencoder(X_train.shape[1], encoding_dim)
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_data=(X_test, X_test))

# Obter as representações codificadas
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Confirmar as formas das saídas codificadas
print(f'X_train_encoded shape: {X_train_encoded.shape}')  # Deve ser (num_samples, 64)
print(f'X_test_encoded shape: {X_test_encoded.shape}')    # Deve ser (num_samples, 64)

# Definir função para criar MLP
def create_mlp(hidden_neurons, input_shape, output_shape):
    model = Sequential()
    model.add(Dense(hidden_neurons, input_dim=input_shape, activation='relu'))  # Camada oculta
    model.add(Dense(output_shape, activation='softmax'))  # Camada de saída (10 classes)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Teste 1: Heurística básica - Regra de Kolmogorov
input_dim = encoding_dim  # Atualizar a dimensão de entrada para o espaço latente do autoencoder
output_dim = 10  # Número de classes (10 dígitos)
neurons_heuristic = 2 * input_dim + 1
model_heuristic = create_mlp(neurons_heuristic, input_dim, output_dim)

# Treinar o modelo MLP com dados codificados
history_heuristic = model_heuristic.fit(X_train_encoded, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test_encoded, y_test_one_hot))

# Avaliar o modelo
loss, accuracy = model_heuristic.evaluate(X_test_encoded, y_test_one_hot)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.3481 - val_loss: 0.1627
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.1529 - val_loss: 0.1266
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.1229 - val_loss: 0.1088
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.1069 - val_loss: 0.0980
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0968 - val_loss: 0.0904
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0901 - val_loss: 0.0853
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0852 - val_loss: 0.0819
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0820 - val_loss: 0.0795
Epoch 9/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0799 - val_loss: 0.0778
Epoch 10/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0781 - val_loss: 0.0766
Epoch 11/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0772 - val_loss: 0.0758
Epoch 12/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/s

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6973 - loss: 0.0426 - val_accuracy: 0.9197 - val_loss: 0.0122
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9139 - loss: 0.0131 - val_accuracy: 0.9250 - val_loss: 0.0115
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9302 - loss: 0.0107 - val_accuracy: 0.9336 - val_loss: 0.0101
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9425 - loss: 0.0088 - val_accuracy: 0.9426 - val_loss: 0.0089
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9501 - loss: 0.0077 - val_accuracy: 0.9387 - val_loss: 0.0093
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9557 - loss: 0.0068 - val_accuracy: 0.9581 - val_loss: 0.0067
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9583 - loss: 0.0065 - val_accuracy: 0.9602 - val_loss: 0.0062
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9620 - loss: 0.0060 - val_accurac

O algoritmo utilizando autoencoder para redução de dimensionalidade + o MLP desenvolvido com os modelos apresentados anteriormente obteve o resultado de precisão de 96,43% para os dados de teste, valor bem próximo ao encontrado utilizando PCA + MLP e melhor do que o apresentado no modelo de comparação da questão 1.

|Modelos|Acurácia|
|-------|--------|
|linear classifier (1-layer NN) - Yann Lecun	|88%|
|Classificador Linear MMQ|86,03%|
|Adaline com Função de Ativação Sigmoidal|86,58%|
|PCA + Classificador linear com Minimos Quadrados|86,14%|
|PCA com modelo Adaline e função de ativação Sigmoidal|88,48%|
|Autoencoder + Adaline c/ Função Sigmoidal|86,76%|
|2-layer NN, 300 hidden units, mean square error - Yann Lecun|95,3%|
|MLP com neuronios ocultos definidos por PCA ou Heuristica ou random search|----|
| Regra do valor médio(Heurística)| 92,75%|
| Regra da raiz quadrada(Heurística)| 89,20%|
| Kolgomorov(Heurística)| 93,58%|
| PCA       | 92,63%|
| Busca Exaustiva (250 neurônios)| 93,47%|
|PCA + MLP + regra de Kolgomorov |96,86%|
|Autoencoder + MLP|96,42%|


Observando todos os modelos desenvolvidos e comparados com os modelos que são base para comparação, em relação ao classificador linear, apenas o modelo PCA + Adaline conseguiu valor superior, os outros modelos tiveram valores próximos mas nao abaixo do modelo de Yann Lecun.

Em relação aos modelos utilizando redes neurais, os modelos PCA + MLP utilizando a quantidade de neurônios da camada oculta usando Kolgomorov e o modelo usando Autoencoder e a regra de Kolgomorov apresentaram resultados superiores ao modelo de redes neurais utilizado como base de comparação.
